# Student

In [1]:
import numpy as np

In [2]:
class World:
    def __init__(self, subj_num, seed=None):
        np.random.seed(seed)
        self.subjs = ["%d+%d=%d" % (i+1,j+1,i+j+2) for i in range(subj_num) for j in range(subj_num)]
    
    def __repr__(self):
        return "%s(%r)" % (self.__class__, self.__dict__)
    
    def random_sim_subjs(self, group_num):
        def split(subjs, n):
            g = [[] for _ in range(n)]
            for s in subjs:
                i = np.random.randint(n)
                g[i].append(s)
            return g

        groups = split(self.subjs, group_num)
        sims = {}
        for group in groups:
            for subj in group:
                sims[subj] = [sim for sim in group if sim!=subj]
        return sims

    def evaluate(self, student, prn=False):
        v = 0
        for subj in self.subjs:
            v += student.skills.get(subj, 0)
        v /= len(self.subjs)
        if prn:
            print(v)
        return v

In [3]:
class BaseTeacher:
    def __init__(self, subjs):
        self.subjs = subjs

    def teach(self, student):
        if type(student) is list:
            for s in student:
                self.teach(s)
        else:
            self._do_teach(student)
    
    def _do_teach(self, student):
        raise NotImplementedError

        
class RandomTeacher(BaseTeacher):
    def _do_teach(self, student):
        subj = np.random.choice(self.subjs)
        student.learn(subj)
        #print(s)

In [4]:
class Student:
    def __init__(self, sim_subjs, learn_rate, forget_rate, sim_rate):
        self.skills = {}
        self.sim_subjs = sim_subjs
        self.lr = learn_rate
        self.fr = forget_rate
        self.sr = sim_rate

    def clear(self):
        for s in self.skills.keys():
            self.skills[s] = 0

    def learn(self, subj):
        # update subj skill
        v = self.skills.get(subj, 0)
        self.skills[subj] = min(v+self.lr, 1) 
        # update similar skills
        for sim in self.sim_subjs.get(subj, []):
            v = self.skills.get(sim, 0)
            self.skills[sim] = min(v+self.lr*self.sr, 1) 

    def forget(self):
        for s,v in self.skills.items():
            self.skills[s] = v*(1-self.fr)
            
    def __repr__(self):
        return "%s(%r)" % (self.__class__, self.__dict__)
   

In [5]:
class Species:
    def __init__(self, sim_subjs, learn_rate, forget_rate, sim_rate):
        self.sim_subjs   = sim_subjs
        self.learn_rate  = learn_rate
        self.forget_rate = forget_rate
        self.sim_rate    = sim_rate
        

In [12]:
wd = World(subj_num=10, seed=None)
tr = RandomTeacher(subjs=wd.subjs)
sp = Species(wd.random_sim_subjs(group_num=10), learn_rate=.1, forget_rate=.01, sim_rate=.5)
st = Student(sp.sim_subjs, sp.learn_rate, sp.forget_rate, sp.sim_rate)

wd.evaluate(st, prn=True)

for _ in range(200):
    tr.teach(st)
    st.forget()
    
wd.evaluate(st, prn=False)


0.0


0.5008509805612397